In [4]:
import awkward as ak
import numpy as np
import pickle as pkl
import hist 
import matplotlib.pyplot as plt
import uproot

import os
from numba import njit

Ultimate goal: implement multiprocessing for analyzing data. 

Structure: have a main function that does analysis, creating histograms and counting Z's. I can then have a separate process for each run of the event generator, which in principle will give me a 4x speed-up. 

For each run, I will need to process the data in batches. This should be fine so long as we aren't doing any mixed event corrections, which we aren't yet. Those would need a separate implementation.

In [8]:
def process_events(parent_data_dir, run_num):
    Z_counter = 0
    W_counter = 0

    events_w_Z = ak.Array([])
    
    run_data_dir = parent_data_dir + f'run{run_num}/' 
    for file in os.listdir(run_data_dir):
        
        with open(run_data_dir + file, 'rb') as in_file:
            events = pkl.load(in_file)
            
            z_counts = np.sum(events['pdg'] == 23)
            W_counts = np.sum(events['pdg'] == abs(24))
            
            if z_counts > 0:
                
                contains_Z = ak.any(events['pdg']==23, axis=1)
                events_w_Z = ak.concatenate([events_w_Z, events[contains_Z]])
            
            Z_counter += z_counts
            W_counter += W_counts
            
    print(Z_counter, W_counter)

In [9]:
RAW_DATA_DIR = '/Users/ravikoka/repos/z_plus_hf/feasibility/data/multi/'
process_events(RAW_DATA_DIR, 1)

0 0
